In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import least_squares

In [2]:
def f(tc, t, beta):
    return (tc-t)**beta
def g(tc, t, beta, omega):
    return f(tc, t, beta)*np.cos(omega*np.log(tc-t))
def h(tc, t, beta, omega):
    return f(tc, t, beta)*np.sin(omega*np.log(tc-t))

In [3]:
def LPPL(t, A, B, C1, C2, beta, omega, tc):
    return A + B*f(tc, t, beta) + C1*g(tc, t, beta, omega) + C2*h(tc, t, beta, omega)

In [4]:
def fit_ABC(beta, omega, tc, log_price):
    one_col = np.ones(len(log_price))

    t = np.arange(1,len(log_price)+1)
    f_col = f(tc, t, beta)
    g_col = g(tc, t, beta, omega)
    h_col = h(tc, t, beta, omega)

    X = np.array([one_col,f_col,g_col,h_col]).T
    A, B, C1, C2 = np.linalg.solve(X.T@X,X.T@log_price)
    return A, B, C1, C2

In [5]:
def f_min( beta, omega, tc, log_price):
    t = np.arange(1,len(log_price)+1)
    A, B, C1, C2 = fit_ABC(beta, omega, tc, log_price)
    return LPPL(t, A, B, C1, C2, beta, omega, tc) - log_price

In [6]:
def opti_config(log_price):
    beta_0 = 0.5
    omega_0 = 9
    tc_0 = len(log_price) + 15

    beta_m = 0.000001 
    omega_m = 1
    tc_m = len(log_price) + 1

    beta_M = 2
    omega_M = 21
    tc_M = len(log_price) + 10000
       
    return beta_0, omega_0, tc_0, beta_m, omega_m, tc_m, beta_M, omega_M, tc_M

In [7]:
def fit_LPPL(log_price):

    beta_0, omega_0, tc_0, beta_m, omega_m, tc_m, beta_M, omega_M, tc_M = opti_config(log_price)

    sol = least_squares(
        lambda x: f_min(beta=x[0], omega=x[1], tc=x[2], log_price=log_price),
        np.array([beta_0, omega_0,  tc_0]),
        bounds = ([beta_m, omega_m, tc_m], [beta_M, omega_M, tc_M])
    )

    if sol.success:
        beta, omega, tc = sol.x
        A, B, C1, C2 = fit_ABC(beta, omega, tc, log_price)
        sol.x = np.append(sol.x,[A, B, C1, C2])
    else:
        print("OLS failed.")
        sol.x = np.append(sol.x,np.zeros(4))
    return sol

In [8]:
def conditions_satisfied(sol, dt):
    beta, omega, tc, A, B, C1, C2 = sol.x

    c1 = 0.01 < beta < 1.2
    c2 = 2 < omega < 25
    c3 = 0.95*dt < tc < 1.11*dt
    c4 = 2.5 < omega/(2*np.pi)*np.log(abs(tc/(tc-dt)))
    C = C1/np.cos(np.arctan(C2/C1))
    c5 = 0.8 < beta * abs(B) / (omega * abs(C))
    return c1 and c2 and c3 and c4 and c5 

In [9]:
def total_return(log_price):
    price = np.array(np.exp(log_price))
    return (price[-1]-price[0])/price[0]

In [56]:
def get_dt_max(sols):
    residuals = [residual(sol) for sol in sols]
    residuals_lgrn = obtainLagrangeRegularizedNormedCost(residuals)
    return np.argmin(residuals_lgrn)

def residual(sol):
    cost = sol.cost
    dt = len(sol.fun)
    return 2*cost/(len(dt)-7)

def obtainLagrangeRegularizedNormedCost(residuals):
    slope = LagrangeMethod(residuals)
    residuals_lgrn = residuals - slope*len(residuals)
    return residuals_lgrn

def LagrangeMethod(residuals):
    slope = calculate_slope_of_normed_cost(residuals)
    return slope[0]

def calculate_slope_of_normed_cost(residuals):
    regr =linear_model.LinearRegression(fit_intercept=False)
    x_residuals = np.arange(len(residuals))
    x_residuals = x_sse.reshape(len(residuals),1)
    res = regr.fit(x_residuals, residuals)
    return res.coef_

In [58]:
def LPPL_confidence(log_price, time_windows):
    
    sols = []
    for dt in time_windows:
        sols.append(fit_LPPL(log_price[-dt:]))

    dt_max = get_dt_max(sols)
    
    LPPL_confidences = []
    total_returns = []
    for dt in time_windows:
        sol = sols.pop(0)
        if dt <= dt_max and conditions_satisfied(sol, dt):
            LPPL_confidences.append(1)
            total_returns.append(total_return(log_price[-dt:]))

    return np.mean(LPPL_confidences)*np.sign(np.median(total_returns))


In [57]:
t

array([750, 550, 350, 150])

In [37]:
sol = fit_LPPL(log_price[-100:])

In [11]:
def LPPL_confidence_signal(log_price, time, time_windows):
    
    LPPL_confidence_ts = []
    for t2 in time:
        LPPL_confidence_ts.append(LPPL_confidence(log_price[:t2], time_windows))
    return pd.DataFrame(LPPL_confidence_ts,index=log_price[time].index).fillna(0)

In [35]:
short_dt = range(30,35)
medium_dt = range(91,300)
long_dt = range(301,750)

price = pd.read_csv("data.csv")["Closing Price (USD)"]
log_price = np.log(price)

LPPL_conf_ts = LPPL_confidence_signal(log_price, time=range(800,2000), time_windows=np.linspace(750,150,4,dtype=int))

OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.
OLS failed.


In [ ]:
plt.plot(log_price[LPPL_conf_ts.index])

In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
ticker = '^GSPC'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2019-01-19'
end_date = '2020-08-10'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
price = panel_data['Adj Close']
log_price = np.log(price)



In [ ]:
panel_data.to_csv('^GSPC_19.csv')

In [ ]:
plt.plot(log_price[150:250])

In [ ]:
panel_data

In [ ]:
len(log_price)

In [ ]:
LPPL_conf_ts = LPPL_confidence_ts(log_price, t2=range(150,250), time_window=np.linspace(150,30,4))
plt.plot(LPPL_conf_ts)